----
----
## Bu uygulamada:

* Gerçek model çalıştırmayacağız,

* Sadece iskelet sınıflarımızı derleyip çalıştıracağız,

* process() çağrıldığında içeride hangi adımların tetiklendiğini loglarla göreceğiz.

Böylece:

**“ImageFrame → PerceptionNode.process() → preprocess → inference → postprocess → DetectedObject listesi”**
zinciri kafaya tam oturacak.

----
----

# Uygulama 1 – “Perception İskeletini Tanıma” Uygulaması
### Amaç

* Proje iskeletini kurmak

* ImageFrame, DetectedObject, PerceptionConfig, PerceptionNode tiplerini gerçek C++ dosyalarıyla görmek

* PerceptionNode::process() akışını console log’larıyla takip etmek

* Henüz LibTorch model çalıştırmadan, sadece node yapısını anlamak

## Zincir tam olarak şu:
```bash
cv::Mat (test.jpg)
     ↓
ImageFrame
     ↓
PerceptionNode::process(frame)
     ↓
   preprocess(frame)
     ↓
   inference(tensor)
     ↓
   postprocess(frame, output)
     ↓
std::vector<DetectedObject>
     ↓
 main.cpp'de ekrana/log'a yazma

### Loglarda bu adımlar tek tek görülecek:

* [PerceptionNode::preprocess] Çalıştı.

* [PerceptionNode::inference] Çalıştı.

* [PerceptionNode::postprocess] Çalıştı.

* “Dummy nesne eklendi.”

* “Toplam tespit edilen nesne sayısı: 1”

### Node yapısının davranışı

#### Uygulama 1 şunu gösteriyor:

* PerceptionNode gerçek bir C++ sınıf olarak ayakta duruyor.

* Dışarıdan sadece process(frame) görünüyor.

* İçeride hangi fonksiyonların hangi sırayla çalıştığı kontrol altında.

* Model yüklü olmasa bile (dummy mod), tasarım zaten hazır.

* Yani: “Bu node dediğimiz şey sadece teori değil, C++’ta çalışan bir nesne.

### Dummy DetectedObject ile “çıktı formatını” anlama

Bu aşamada:

* Gerçek YOLO sonucu yok,

Ama DetectedObject tipinin ne içerdiğini görüyorsun:

* class_id, class_name, score

* bbox (x, y, width, height)

main.cpp bunu ekrana bastığında, artık:

**“Benim algılama modülüm dışarıya şu formatta veri veriyor.”**

bilgisi kesinleşmiş oluyor.

### Nereye ne eklenecek sorusunun cevabını görmek

Uygulama 1, ileride yapılacak işleri kafanda şu şekilde konumlandırman için:

* preprocess() → burada cv::Mat → Tensor dönüşümü yazılacak.

* inference() → burada model_.forward(...) kodu yazılacak.

* postprocess() → burada YOLO/CNN çıktısının decode edilmesi (bbox, NMS, class) yazılacak.

* Şu an hepsi iskele halinde. Yani ileride parça parça dolduracağın yerler belli.

-----

# ⭐ Uygulama 1: “PerceptionNode İskeletini Tanıma”

### Bu uygulamayı neden yapıyoruz?
**Çünkü gerçek YOLO/CNN entegrasyonuna geçmeden önce node mimarisinin nasıl çalıştığını görmemiz gerekiyor.**
Sonraki tüm uygulamaların temeli bu yapı olacak.

## ⭐ Uygulama 1’in nihai hedefi

* Uygulamayı çalıştırdığında terminalde şuna benzeyen bir çıktı görmelisin:
```bash
[PerceptionNode] CPU kullanılacak.
[PerceptionNode] model_path boş, model yüklenmeyecek (Uygulama 1 modu).

========== PerceptionNode::process ==========
Frame ID  : 1
Camera    : front_camera
Image size: 1920 x 1080

[PerceptionNode::preprocess] Çalıştı.
[PerceptionNode::inference] Çalıştı.
[PerceptionNode::postprocess] Çalıştı.
[PerceptionNode::postprocess] Dummy nesne eklendi.
Toplam tespit edilen nesne sayısı: 1
=============================================

[main] Dönüşte gelen tespitler:
class_id: 0, name: dummy_object, score: 0.99, bbox: (480, 270, 960, 540)
```

### Bu çıktı sana şunu öğretmiş olacak:

* PerceptionNode dediğimiz yapı tek bir fonksiyon çağrısıyla komple bir algılama pipeline'ı çalıştırıyor.
* Bu pipeline’ın adımları bağımsız, modüler ve birbirine bağlı şekilde çalışıyor.


----

## Son olarak :

### .hpp Uzantısı Nedir?

* .hpp dosyası C++ header (başlık) dosyasıdır.

* Aynı .h dosyalarına benzer ama özellikle C++ kodlarında kullanıldığı anlamına gelir.

### .hpp ile .h arasındaki fark nedir?

| Uzantı   | Kullanım Alanı                                                  |
| -------- | --------------------------------------------------------------- |
| **.h**   | C ve C++ için kullanılabilir (genel header dosyası)             |
| **.hpp** | Özellikle C++ projeleri için kullanılır; class, template içerir |

* Yani .hpp, “bu başlık dosyası tamamen C++” demektir.

Bu ayrım özellikle büyük projelerde düzeni korumak için kullanılır.

-----
---

### Artık kod kısmına geçiş yapabiliriz.

-----
-----

## 1️⃣ src/perception_types.hpp

```cpp
#ifndef PERCEPTION_TYPES_HPP
#define PERCEPTION_TYPES_HPP

#include <cstdint>
#include <string>
#include <vector>
#include <chrono>

#include <opencv2/core.hpp>

// Kameradan gelen tek kare + bağlam bilgisi
struct ImageFrame
{
    cv::Mat image;  // BGR, 8-bit

    std::uint64_t frame_id{0};

    std::chrono::steady_clock::time_point timestamp{
        std::chrono::steady_clock::now()
    };

    std::string camera_name;

    ImageFrame() = default;

    ImageFrame(const cv::Mat& img,
               std::uint64_t id,
               const std::string& cam)
        : image(img)
        , frame_id(id)
        , timestamp(std::chrono::steady_clock::now())
        , camera_name(cam)
    {}
};

// Tek bir tespit edilen nesne
struct DetectedObject
{
    int class_id{-1};
    std::string class_name;
    float score{0.0f};

    // sol-üst köşe + genişlik / yükseklik (piksel)
    float x{0.0f};
    float y{0.0f};
    float width{0.0f};
    float height{0.0f};

    int track_id{-1};
};

// Node konfigürasyonu
struct PerceptionConfig
{
    std::string model_path;       // Uygulama 1'de boş kalabilir
    std::string device{"cpu"};    // "cpu" veya "cuda"

    int input_width{640};
    int input_height{640};

    // Normalize için mean/std (RGB)
    std::vector<float> mean{0.0f, 0.0f, 0.0f};
    std::vector<float> std{1.0f, 1.0f, 1.0f};

    // Tespit eşikleri (ileride YOLO için)
    float score_threshold{0.25f};
    float nms_iou_threshold{0.45f};

    std::vector<std::string> class_names;
};

#endif // PERCEPTION_TYPES_HPP


## 2️⃣ src/perception_node.hpp

```cpp
#ifndef PERCEPTION_NODE_HPP
#define PERCEPTION_NODE_HPP

#include "perception_types.hpp"

#include <torch/script.h>
#include <iostream>

class PerceptionNode
{
public:
    explicit PerceptionNode(const PerceptionConfig& config);

    // Dışarıya açılan ana API
    std::vector<DetectedObject> process(const ImageFrame& frame);

private:
    void loadModel(const std::string& path);

    torch::Tensor preprocess(const ImageFrame& frame);
    torch::Tensor inference(const torch::Tensor& input);
    std::vector<DetectedObject> postprocess(const ImageFrame& frame,
                                            const torch::Tensor& output);

private:
    PerceptionConfig config_;
    torch::jit::script::Module model_;
    torch::Device device_{torch::kCPU};
    bool model_loaded_{false};
};

#endif // PERCEPTION_NODE_HPP


## 3️⃣ src/perception_node.cpp

```cpp
#include "perception_node.hpp"

PerceptionNode::PerceptionNode(const PerceptionConfig& config)
    : config_(config)
{
    // Cihaz seçimi
    if (config_.device == "cuda" && torch::cuda::is_available())
    {
        device_ = torch::kCUDA;
        std::cout << "[PerceptionNode] CUDA kullanılacak.\n";
    }
    else
    {
        device_ = torch::kCPU;
        std::cout << "[PerceptionNode] CPU kullanılacak.\n";
    }

    // Uygulama 1: Model yükleme zorunlu değil
    if (!config_.model_path.empty())
    {
        loadModel(config_.model_path);
    }
    else
    {
        std::cout << "[PerceptionNode] model_path boş, model yüklenmeyecek (Uygulama 1 modu).\n";
    }
}

void PerceptionNode::loadModel(const std::string& path)
{
    try
    {
        std::cout << "[PerceptionNode] Model yükleniyor: " << path << "\n";
        model_ = torch::jit::load(path, device_);
        model_.eval();
        model_loaded_ = true;
        std::cout << "[PerceptionNode] Model yüklendi.\n";
    }
    catch (const c10::Error& e)
    {
        std::cerr << "[PerceptionNode] Model yükleme hatası: " << e.what() << "\n";
        model_loaded_ = false;
    }
}

std::vector<DetectedObject> PerceptionNode::process(const ImageFrame& frame)
{
    std::cout << "========== PerceptionNode::process ==========\n";
    std::cout << "Frame ID  : " << frame.frame_id << "\n";
    std::cout << "Camera    : " << frame.camera_name << "\n";
    std::cout << "Image size: " << frame.image.cols << " x " << frame.image.rows << "\n";

    // 1) Preprocess
    auto input_tensor = preprocess(frame);

    // 2) Inference
    auto output_tensor = inference(input_tensor);

    // 3) Postprocess
    auto detections = postprocess(frame, output_tensor);

    std::cout << "Toplam tespit edilen nesne sayısı: " << detections.size() << "\n";
    std::cout << "=============================================\n";

    return detections;
}

torch::Tensor PerceptionNode::preprocess(const ImageFrame& frame)
{
    std::cout << "[PerceptionNode::preprocess] Çalıştı.\n";

    if (frame.image.empty())
    {
        std::cerr << "[PerceptionNode::preprocess] Uyarı: image boş.\n";
    }

    std::cout << "[PerceptionNode::preprocess] Hedef input size: "
              << config_.input_width << " x " << config_.input_height << "\n";

    // Uygulama 1: Sadece dummy tensor döndürüyoruz.
    // İleride burada cv::Mat → torch::Tensor dönüşümü yapılacak.
    return torch::empty({0}, device_);
}

torch::Tensor PerceptionNode::inference(const torch::Tensor& input)
{
    std::cout << "[PerceptionNode::inference] Çalıştı.\n";

    if (!model_loaded_)
    {
        std::cout << "[PerceptionNode::inference] Model yüklü değil. Dummy çıktı döndürülüyor.\n";
        return torch::empty({0}, device_);
    }

    torch::NoGradGuard no_grad;

    // Uygulama 1: Gerçek model çağrısını sonraki uygulamalara bırakıyoruz.
    std::cout << "[PerceptionNode::inference] (Gerçek model çağrısı Uygulama 2/3'te eklenecek.)\n";

    return torch::empty({0}, device_);
}

std::vector<DetectedObject> PerceptionNode::postprocess(const ImageFrame& frame,
                                                        const torch::Tensor& output)
{
    std::cout << "[PerceptionNode::postprocess] Çalıştı.\n";

    std::vector<DetectedObject> result;

    if (frame.image.empty())
    {
        std::cout << "[PerceptionNode::postprocess] Boş frame, tespit yok.\n";
        return result;
    }

    // Uygulama 1: Sadece 1 adet sahte tespit döndürüyoruz.
    DetectedObject obj;
    obj.class_id = 0;
    obj.class_name = "dummy_object";
    obj.score = 0.99f;

    // Görüntünün ortasına yakın bir bbox örneği
    obj.x = frame.image.cols * 0.25f;
    obj.y = frame.image.rows * 0.25f;
    obj.width  = frame.image.cols * 0.5f;
    obj.height = frame.image.rows * 0.5f;

    result.push_back(obj);

    std::cout << "[PerceptionNode::postprocess] Dummy nesne eklendi.\n";

    return result;
}


## 4️⃣ src/main.cpp

```cpp

#include <iostream>
#include <opencv2/opencv.hpp>

#include "perception_types.hpp"
#include "perception_node.hpp"

int main()
{
    // 1) Test resmi yükle
    cv::Mat img = cv::imread("test.jpg");
    if (img.empty())
    {
        std::cerr << "[main] test.jpg yüklenemedi. Çalışma klasörünü ve yolu kontrol et.\n";
        return 1;
    }

    // 2) ImageFrame oluştur
    ImageFrame frame(img, /*frame_id=*/1, /*camera_name=*/"front_camera");

    // 3) Config hazırla (Uygulama 1 için model_path boş bırakılabilir)
    PerceptionConfig config;
    config.device = "cpu";
    config.input_width = 640;
    config.input_height = 640;

    // 4) PerceptionNode oluştur
    PerceptionNode perception(config);

    // 5) process() çağır
    auto detections = perception.process(frame);

    // 6) Sonuçları yazdır
    std::cout << "\n[main] Dönüşte gelen tespitler:\n";
    for (const auto& obj : detections)
    {
        std::cout << "  class_id: " << obj.class_id
                  << ", name: " << obj.class_name
                  << ", score: " << obj.score
                  << ", bbox: (" << obj.x << ", " << obj.y
                  << ", " << obj.width << ", " << obj.height << ")\n";
    }

    return 0;
}


## 5️⃣ (Opsiyonel) Minimal CMakeLists.txt

```bash
cmake_minimum_required(VERSION 3.18)
project(PerceptionProject CXX)

set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED ON)

# OpenCV
find_package(OpenCV REQUIRED)

# LibTorch (TORCH_DIR'i daha önce set etmiş olman gerekiyor)
find_package(Torch REQUIRED)

add_executable(perception_app
    src/main.cpp
    src/perception_types.hpp
    src/perception_node.hpp
    src/perception_node.cpp
)

target_link_libraries(perception_app
    ${OpenCV_LIBS}
    ${TORCH_LIBRARIES}
)

# Windows için gerekebilir
if (MSVC)
    target_compile_options(perception_app PRIVATE /bigobj)
endif()


---